In [1]:
import os, sys, mne, socket, time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nilearn import plotting
from itertools import product

from initialization import load_paths
from preprocFuncs import getRawData, getTrials, runTGA, runTGA_TMS
# from plotters import plotPower, plotRSA, plotDiffRSA
from decodeHelpers import *

In [2]:
subjID = 10
master_df =  pd.read_csv('/d/DATD/datd/MD_TMS_EEG/analysis/meta_analysis/master_df_calib.csv')
notmsdays = master_df[(master_df['subjID'] == subjID) & (master_df['istms']==1)]['day'].unique()[0]
days_valid = [1, 2, 3]
tmsdays = [day for day in days_valid if day != notmsdays]

for idx, day in enumerate(tmsdays):

    p = load_paths(subjID, day)
    # Load behavioral data
    behav_df = master_df[(master_df['subjID'] == subjID) & (master_df['day'] == day)]
    # Load raw data
    raw_data = getRawData(p)

    # Define events
    events, tah = mne.events_from_annotations(raw_data)
    events_dict = {
        'BlockStart': 1001,
        'Fixation': 1,
        'Delay1': 2,
        'Delay2': 3,
        'Response': 4,
        'Feedback': 6,
        'ITI': 7,
        'BlockEnd': 8,
    }
    trl_events = {
        'pro_inPF': 11,
        'pro_outPF': 12,
        'anti_inPF': 13,
        'anti_outPF': 14
    }

    # Define electrodes of interest
    left_occ_elecs = ['O1', 'PO3', 'PO7', 'P1', 'P3', 'P7']
    right_occ_elecs = ['O2', 'PO4', 'PO8', 'P2', 'P4', 'P8']
    left_par_elecs = ['P7', 'P5', 'P3', 'CP5', 'CP3', 'CP1', 'C5', 'C3', 'C1']
    right_par_elecs = ['P8', 'P6', 'P4', 'CP6', 'CP4', 'CP2', 'C6', 'C4', 'C2']

    left_elecs = ['O1', 'PO3', 'PO7', 'P1', 'P3', 'P7', 'P5', 'CP5', 'CP3', 'CP1', 'C5', 'C3', 'C1']
    right_elecs = ['O2', 'PO4', 'PO8', 'P2', 'P4', 'P8', 'P6', 'CP6', 'CP4', 'CP2', 'C6', 'C4', 'C2']

    left_occ_elecs = [elec for elec in left_occ_elecs if elec in raw_data.ch_names]
    right_occ_elecs = [elec for elec in right_occ_elecs if elec in raw_data.ch_names]
    left_par_elecs = [elec for elec in left_par_elecs if elec in raw_data.ch_names]
    right_par_elecs = [elec for elec in right_par_elecs if elec in raw_data.ch_names]
    left_elecs = [elec for elec in left_elecs if elec in raw_data.ch_names]
    right_elecs = [elec for elec in right_elecs if elec in raw_data.ch_names]

    # Epoch data
    if idx == 0:
        epochData1 = mne.Epochs(raw_data, events, event_id=trl_events, tmin=-1, tmax=4.5, baseline=None, preload=True)
        epochDataBasecorr1 = mne.Epochs(raw_data, events, event_id=trl_events, tmin=-1, tmax=4.5, baseline=(-1, 0), preload=True)
    else:
        epochData2 = mne.Epochs(raw_data, events, event_id=trl_events, tmin=-1, tmax=4.5, baseline=None, preload=True)
        epochDataBasecorr2 = mne.Epochs(raw_data, events, event_id=trl_events, tmin=-1, tmax=4.5, baseline=(-1, 0), preload=True)


Loading pre-existing data ...
Opening raw data file /d/DATD/datd/MD_TMS_EEG/EEGpy/sub10/day01/sub10_day01_raw.fif...
    Range : 0 ... 4282399 =      0.000 ...  4282.399 secs
Ready.
Reading 0 ... 4282399  =      0.000 ...  4282.399 secs...
Non-RawBrainVision raw using branvision markers
Used Annotations descriptions: ['Response/R  1', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  6', 'Stimulus/S  7', 'Stimulus/S  8', 'Stimulus/S 11', 'Stimulus/S 12', 'Stimulus/S 13', 'Stimulus/S 14']
Not setting metadata
400 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 400 events and 5501 original time points ...
0 bad epochs dropped
Not setting metadata
400 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 400 events and 5501 original time points ...
0 bad epochs dropped
Loading pre-existing data ...
Opening raw data file

In [4]:
epochData2

Number of events,400
Events,anti_inPF: 100anti_outPF: 100pro_inPF: 100pro_outPF: 100
Time range,-1.000 – 4.500 s
Baseline,off


In [5]:
elecs = left_occ_elecs+right_occ_elecs
freq_band = 'alpha'
POWPHASE = ['pow', 'phase']
TYPECOND = ['byPF_pro', 'byPF_anti', 'byPF_stimin', 'byPF_respin', 'byTrlType']

for pow_or_phase, typeCond in product(POWPHASE, TYPECOND):
    scores_mean, t_down = runTGA_TMS(epochData1, epochData2, elecs, freq_band, pow_or_phase, typeCond)
    np.savez(f"{p['EEGroot']}_{pow_or_phase}_{typeCond}.npy", scores_mean=scores_mean, t_array_down=t_down)

Setting up band-pass filter from 8 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 1651 samples (1.651 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 18960 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 49288 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 50400 out of 50400 | elapsed:   14.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]




  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]
  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]



 35%|███▍      | Fitting GeneralizingEstimator : 38/109 [00:00<00:00,  578.18it/s]A


 19%|█▉        | Fitting GeneralizingEstimator : 21/109 [00:00<00:00,  424.47it/s]

  6%|▌         | Fitting GeneralizingEstimator : 6/109 [00:00<00:00,  118.48it/s]

 72%|███████▏  | Fitting Ge

Setting up band-pass filter from 8 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 1651 samples (1.651 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 2040 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 18936 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 48782 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 50400 out of 50400 | elapsed:   16.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]


  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]

 12%|█▏        | Fitting GeneralizingEstimator : 13/109 [00:00<00:00,  118.17it/s]


 16%|█▌        | Fitting GeneralizingEstimator : 17/109 [00:00<00:00,  162.98it/s]

 29%|██▉       | Fitting GeneralizingEstimator : 32/109 [00:00<00:00,  215.33it/s]


 38%|███▊      | Fitting GeneralizingEstimator : 41/109 [00:00<00:00,  283.76it/s]

 53%|█████▎    | Fitt

Setting up band-pass filter from 8 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 1651 samples (1.651 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 2040 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 25080 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 49624 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 50400 out of 50400 | elapsed:   17.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]
  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]

 14%|█▍        | Fitting GeneralizingEstimator : 15/109 [00:00<00:00,  140.86it/s]A
 24%|██▍       | Fitting GeneralizingEstimator : 26/109 [00:00<00:00,  264.22it/s]


 39%|███▊      | Fitting GeneralizingEstimator : 42/109 [00:00<00:00,  274.62it/s]A

 15%|█▍        | Fitting GeneralizingEstimator : 16/109 [00:00<00:00,  138.46it/s]
 38%|███▊      | Fitting G

Setting up band-pass filter from 8 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 1651 samples (1.651 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 2040 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 13560 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 29688 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 49668 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 50400 out of 50400 | elapsed:   16.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]

  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]
  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]
  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]


  0%|          | Fitting GeneralizingEstimator : 0/109 [00:00<?,       ?it/s]




  0%|          | Fitting GeneralizingEstimator : 0/109 [00:

ValueError: The number of classes has to be greater than one; got 1 class